数据集来源：使用make_data_dota.py得到DOTA中含有ship的图像。本项目Dataset_cyclegan利用标签把可见光中的ship替换红外ship（背景保持不变），使构建的训练图像对中两张图的差别仅是标签处的船不同。与v3的区别是抠掉的位置不一定在红外的标签处，而是随机  
修改Dataset中的image_b，是否配对

In [ ]:
import torch as T 
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision.transforms as trans
from torch.utils.data import Dataset, DataLoader
import numpy as np 
from PIL import Image, ImageFilter 
import matplotlib.pyplot as plt 
import os 
import time 

# Global Variable

In [ ]:
# data 
path_a_line = ["/home/wwang/datasets/ship_line/"]
path_a_background = ["/home/wwang/datasets/ship_background/"]
path_b = ["/home/wwang/datasets/ship/data/images/"]
image_size = [256, 256]
# network
channel_d = 64
channel_g = 64
layer_resblock = 9
# training
λ_cycle_consistency = 10 # 10
λ_identity = 5 # 5
batch_size = 1 # 1
num_workers = 4
iterations = 10000000
learning_rate_d1 = 1e-4 # 2e-4
learning_rate_g1 = 1e-4 # 2e-4
learning_rate_d2 = 1e-4 # 2e-4
learning_rate_g2 = 1e-4 # 2e-4
apply_train_combined = False # True 
apply_contrast_loss = False # False 
apply_identity_loss = True # True 
step_g = 1 # 1
pool_size = 50
# 
device = T.device("cuda:0")
path_work = "/home/wwang/wwgeneration/work/CycleGAN_v10/"

# Data

In [ ]:
def path_to_dir(path):
    list_ = []
    for path_ in path:
        dirs = os.listdir(path_)
        dirs.sort()
        for dir_ in dirs:
            list_.append(path_ + dir_)
    return list_


list_a_line = path_to_dir(path_a_line)
list_a_background = path_to_dir(path_a_background)
list_b = path_to_dir(path_b)

In [ ]:
class Dataset_cyclegan(Dataset):
    #
    def __init__(self, list_a_line=list_a_line, list_a_background=list_a_background, list_b=list_b):
        self.list_a_line = list_a_line
        self.list_a_background = list_a_background
        self.list_b = list_b
        self.transform = trans.Compose([trans.ToTensor(),
                                        trans.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                        ])
    #
    def augmentation_image(self, image):
        if np.random.randint(2):
            image = image.transpose(Image.FLIP_LEFT_RIGHT) 
        n = np.random.randint(4)
        if n == 1:
            image = image.rotate(90, expand=1)
        if n == 2:
            image = image.rotate(180, expand=1)
        if n == 3:
            image = image.rotate(270, expand=1)
        return image
    #
    def augmentation_images(self, a, b):
        if np.random.randint(2):
            a = a.transpose(Image.FLIP_LEFT_RIGHT) 
            b = b.transpose(Image.FLIP_LEFT_RIGHT) 
        n = np.random.randint(4)
        if n == 1:
            a = a.rotate(90, expand=1)
            b = b.rotate(90, expand=1)
        if n == 2:
            a = a.rotate(180, expand=1)
            b = b.rotate(180, expand=1)
        if n == 3:
            a = a.rotate(270, expand=1)
            b = b.rotate(270, expand=1)
        return a, b
    #
    def load_imageb(self):
        idx_b = np.random.randint(len(self.list_b))
        image = Image.open(self.list_b[idx_b]).convert('RGB').resize(image_size)
        image = self.augmentation_image(image)
        return image
    #
    def make_images(self):
        #
        idx_b = np.random.randint(len(self.list_b))
        image_b = Image.open(self.list_b[idx_b]).convert('RGB').resize(image_size)
        # image = self.augmentation_image(image)
        #
        idx_line = np.random.randint(len(self.list_a_line))
        image_line = Image.open(self.list_a_line[idx_line])
        image_line = self.augmentation_image(image_line)
        #
        image_background = Image.open(self.list_a_background[idx_b]).convert('RGB').resize(image_size)
        # image_background = self.augmentation_image(image_background)
        #
        image_background, image_b = self.augmentation_images(image_background, image_b)
        #
        w, h = image_background.size
        w_patch, h_patch = np.random.randint(image_size[1]//8, image_size[1]+1), np.random.randint(image_size[0]//8, image_size[0]+1)
        w_random, h_random = np.random.randint(w-w_patch+1), np.random.randint(h-h_patch+1)
        image_background.paste(image_line.resize([w_patch, h_patch]), [w_random, h_random])
        return image_background, image_b
    #
    def __getitem__(self, idx):
        # image_a, image_b = self.make_images()
        image_a, _ = self.make_images()
        image_b = self.load_imageb() # 
        image_a = self.transform(image_a)
        image_b = self.transform(image_b)
        return image_a, image_b
    #
    def __len__(self):
        return iterations

In [ ]:
dataloader = DataLoader(Dataset_cyclegan(), batch_size=1)
n = 0
for image_a, image_b in dataloader:
    if n == 2:
        break
    image_a = image_a.numpy().squeeze().transpose([1,2,0])
    image_b = image_b.numpy().squeeze().transpose([1,2,0])
    plt.subplot(1,2,1), plt.imshow(image_a/2+0.5), plt.title('a')
    plt.subplot(1,2,2), plt.imshow(image_b/2+0.5), plt.title('b')
    plt.show()
    n += 1

# Network Structure

## Discriminator

In [ ]:
class CNA(nn.Module):
    #
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, conv='conv', norm='in', act='relu'):
        super().__init__()
        if conv == 'conv':
            self.layers = nn.ModuleList([nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)])
        if conv == 'tconv':
            self.layers = nn.ModuleList([nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)])
        if norm == 'in':
            self.layers.append(nn.InstanceNorm2d(out_channels))
        if act == 'relu':
            self.layers.append(nn.ReLU())
        if act == 'lrelu':
            self.layers.append(nn.LeakyReLU(0.2))
    #
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class Discriminator(nn.Module):
    #
    def __init__(self, channel=channel_d):
        super().__init__()
        self.layers = nn.ModuleList()
        channels = [3, channel, channel*2, channel*4, channel*8]
        strides = [2, 2, 2, 1]
        norms = [False, 'in', 'in', 'in']
        for i in range(4):
            self.layers.append(CNA(channels[i], channels[i+1], 4, stride=strides[i], padding=1, conv='conv', norm=norms[i], act='lrelu'))
        self.layers.append(nn.Conv2d(channels[i+1], 1, 4, stride=1, padding=1))
    #
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x 

## Generator

In [ ]:
class ResBlock(nn.Module):
    #
    def __init__(self, channel):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, 1, padding_mode='reflect', bias=False),
            nn.InstanceNorm2d(channel),
            nn.ReLU(),
            nn.Conv2d(channel, channel, 3, 1, 1, padding_mode='reflect', bias=False),
            nn.InstanceNorm2d(channel))
    #
    def forward(self, x):
        y = self.layers(x)
        return x + y

In [ ]:
class Generator(nn.Module):
    #
    def __init__(self, channel=channel_g, layer_resblock=layer_resblock):
        super().__init__()
        self.layers = nn.ModuleList()
        channels = [3, channel, channel*2, channel*4]
        kernel_sizes = [7, 4, 4]
        strides = [1, 2, 2]
        paddings = [3, 1, 1]
        for i in range(3):
            self.layers.append(CNA(channels[i], channels[i+1], kernel_sizes[i], strides[i], paddings[i], 'conv', 'in', act='relu'))
        for _ in range(layer_resblock):
            self.layers.append(ResBlock(channels[-1]))
        for i in range(2):
            self.layers.append(CNA(channels[-i-1], channels[-i-2], kernel_sizes[-i-1], strides[-i-1], paddings[-i-1], 'tconv', 'in', 'relu'))
        self.layers.append(nn.ConvTranspose2d(channels[1], channels[0], kernel_size=7, stride=1, padding=3, bias=False))
    #
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return F.tanh(x)

# Show

In [ ]:
def image_translation(image_a, G1, G2):
    #
    # image_a = image.transpose([1,2,0]) * 0.5 + 0.5
    # x = T(image_a[None,:,:,:] * 2 - 1).transpose([0,3,1,2])
    #
    image_a2b = G1(image_a)
    image_a2b2a = G2(image_a2b)
    image_a = image_a.squeeze().cpu().numpy().transpose([1,2,0]) / 2 + 0.5
    image_a2b = image_a2b.squeeze().cpu().numpy().transpose([1,2,0]) / 2 + 0.5
    image_a2b2a = image_a2b2a.squeeze().cpu().numpy().transpose([1,2,0]) / 2 + 0.5
    #
    plt.figure(figsize=[20,20])
    plt.subplot(1,3,1), plt.imshow(image_a), plt.title('a')
    plt.subplot(1,3,2), plt.imshow(image_a2b), plt.title('a2b')
    plt.subplot(1,3,3), plt.imshow(image_a2b2a), plt.title('a2b2a')
    plt.show()

In [ ]:
@T.no_grad()
def show(G1=None, G2=None):
    #
    if G1 is None:
        G1 = Generator().to(device)
    G1.eval()
    if G2 is None:
        G2 = Generator().to(device)
    G2.eval()
    #
    dataloader = DataLoader(Dataset_cyclegan(), batch_size=1)
    for image_a, image_b in dataloader:
        image_a, image_b = image_a.to(device), image_b.to(device)
        image_translation(image_a, G1, G2)
        image_translation(image_b, G2, G1)
        break

In [ ]:
show()

# Training

In [ ]:
class ImagePool():
    #
    def __init__(self, pool_size=pool_size):
        self.pool = []
        self.count = 0
        self.pool_size = pool_size
    #
    def pool_image(self, image):
        if self.count < self.pool_size:
            self.pool.append(image.cpu().numpy())
            self.count += 1
            return image
        else:
            if np.random.randint(2):
                return image
            else:
                random_id = np.random.randint(self.pool_size)
                temp = self.pool[random_id]
                self.pool[random_id] = image.cpu().numpy()
                return T.from_numpy(temp).to(device)

In [ ]:
pool_a2b, pool_b2a = ImagePool(), ImagePool()

In [ ]:
def trainer(net_g1, net_d1, net_g2, net_d2, optimizer_g1, optimizer_d1, optimizer_g2, optimizer_d2, image_a, image_b, apply_identity_loss=apply_identity_loss, apply_contrast_loss=apply_contrast_loss):
    # outputs of generators
    image_a2b = net_g1(image_a)
    image_a2b2a = net_g2(image_a2b)
    image_b2a = net_g2(image_b)
    image_b2a2b = net_g1(image_b2a)
    if apply_identity_loss:
        image_a2a = net_g2(image_a)
        image_b2b = net_g1(image_b)
    # outputs of discriminators
    d_a2b = net_d1(pool_a2b.pool_image(image_a2b))
    d_b = net_d1(image_b)
    d_b2a = net_d2(pool_b2a.pool_image(image_b2a))
    d_a = net_d2(image_a)
    if apply_contrast_loss:
        d_a = net_d1(image_a)
        d_b = net_d2(image_b)
    # loss d1
    loss_d1 = T.mean(d_a2b**2) + T.mean((d_b-1)**2)
    if apply_contrast_loss:
        loss_d1 += T.mean(d_a**2)
        loss_d1 /= 3
    else:
        loss_d1 /= 2
    # loss g1
    loss_g1 = T.mean((d_a2b-1)**2) + (T.mean(T.abs(image_a-image_a2b2a)) + T.mean(T.abs(image_b-image_b2a2b))) * λ_cycle_consistency
    if apply_identity_loss:
        loss_g1 += T.mean(T.abs(image_b-image_b2b)) * λ_identity
    # loss d2
    loss_d2 = T.mean(d_b2a**2) + T.mean((d_a-1)**2)
    if apply_contrast_loss:
        loss_d2 += T.mean(d_b**2)
        loss_d2 /= 3
    else:
        loss_d2 /= 2
    # loss g2
    loss_g2 = T.mean((d_b2a-1)**2) + (T.mean(T.abs(image_b-image_b2a2b)) + T.mean(T.abs(image_a-image_a2b2a))) * λ_cycle_consistency
    if apply_identity_loss:
        loss_g2 += T.mean(T.abs(image_a-image_a2a)) * λ_identity
    # optimizer d1
    optimizer_d1.zero_grad()
    loss_d1.backward(retain_graph=True)
    optimizer_d1.step()
    # optimizer g1
    optimizer_g1.zero_grad()
    loss_g1.backward(retain_graph=True)
    optimizer_g1.step()
    # optimizer d2
    optimizer_d2.zero_grad()
    loss_d2.backward(retain_graph=True)
    optimizer_d2.step()
    # optimizer g2
    optimizer_g2.zero_grad()
    loss_g2.backward()
    optimizer_g2.step()

In [ ]:
def trainer_d(net_g, net_d, optimizer_d, image_a, image_b, pool, apply_contrast_loss=apply_contrast_loss):
    # outputs of generator
    image_a2b = net_g(image_a)
    # outputs of discriminator
    d_a2b = net_d(pool.pool_image(image_a2b.detach()))
    d_b = net_d(image_b)
    if apply_contrast_loss:
        d_a = net_d(image_a)
    # loss d
    loss_d = T.mean(d_a2b**2) + T.mean((d_b-1)**2)
    if apply_contrast_loss:
        loss_d += T.mean(d_a**2)
        loss_d /= 3
    else:
        loss_d /= 2
    # optimizer d
    optimizer_d.zero_grad()
    loss_d.backward()
    optimizer_d.step()

In [ ]:
def trainer_g(net_g1, net_d1, net_g2, optimizer_g1, image_a, image_b, apply_cycle_loss):
    # outputs of generators
    image_a2b = net_g1(image_a)
    image_b2a = net_g2(image_b)
    if apply_cycle_loss:
        image_a2b2a = net_g2(image_a2b)
        image_b2a2b = net_g1(image_b2a)
    if apply_identity_loss:
        image_b2b = net_g1(image_b)
    # outputs of discriminators
    d_a2b = net_d1(image_a2b)
    # loss g1
    loss_g1 = T.mean((d_a2b-1)**2)
    if apply_cycle_loss:
        loss_g1 += (T.mean(T.abs(image_a-image_a2b2a)) + T.mean(T.abs(image_b-image_b2a2b))) * λ_cycle_consistency
    if apply_identity_loss:
        loss_g1 += T.mean(T.abs(image_b-image_b2b)) * λ_identity
    # optimizer g1
    optimizer_g1.zero_grad()
    loss_g1.backward()
    optimizer_g1.step()

In [ ]:
def train(iterations, load_model):
    #
    iteration = np.load(path_work + 'iteration.npy', allow_pickle=True).item() if load_model else -1
    print('Start training from iteration ', str(iteration))
    net_g1 = Generator().to(device)
    net_d1 = Discriminator().to(device)
    net_g2 = Generator().to(device)
    net_d2 = Discriminator().to(device)
    net_d1.train()
    net_d2.train()
    optimizer_g1 = T.optim.Adam(net_g1.parameters(), learning_rate_g1, betas=(0.5, 0.999))
    optimizer_d1 = T.optim.Adam(net_d1.parameters(), learning_rate_d1, betas=(0.5, 0.999))
    optimizer_g2 = T.optim.Adam(net_g2.parameters(), learning_rate_g2, betas=(0.5, 0.999))
    optimizer_d2 = T.optim.Adam(net_d2.parameters(), learning_rate_d2, betas=(0.5, 0.999))
    if load_model:
        net_g1.load_state_dict(T.load(path_work + 'net_g1.pt'))
        optimizer_g1.load_state_dict(T.load(path_work + 'optimizer_g1.pt'))
        net_d1.load_state_dict(T.load(path_work + 'net_d1.pt'))
        optimizer_d1.load_state_dict(T.load(path_work + 'optimizer_d1.pt'))
        net_g2.load_state_dict(T.load(path_work + 'net_g2.pt'))
        optimizer_g2.load_state_dict(T.load(path_work + 'optimizer_g2.pt'))
        net_d2.load_state_dict(T.load(path_work + 'net_d2.pt'))
        optimizer_d2.load_state_dict(T.load(path_work + 'optimizer_d2.pt'))
    
    time_start = time.time()
    dataloader = DataLoader(Dataset_cyclegan(), batch_size=batch_size, shuffle=False, num_workers=num_workers)
    for image_a, image_b in dataloader:
        if iteration >= iterations:
            break
        iteration += 1
        image_a, image_b = image_a.to(device), image_b.to(device)
        if apply_train_combined:
            net_g1.train()
            net_g2.train()  
            trainer(net_g1, net_d1, net_g2, net_d2, optimizer_g1, optimizer_d1, optimizer_g2, optimizer_d2, image_a, image_b)
        else:
            trainer_d(net_g1, net_d1, optimizer_d1, image_a, image_b, pool_a2b)
            trainer_d(net_g2, net_d2, optimizer_d2, image_b, image_a, pool_b2a)
            if iteration % step_g == 0:
                net_g1.train()
                net_g2.train()         
                trainer_g(net_g1, net_d1, net_g2, optimizer_g1, image_a, image_b, apply_cycle_loss=True)
                trainer_g(net_g2, net_d2, net_g1, optimizer_g2, image_b, image_a, apply_cycle_loss=True)
    #
        if iteration % 1000 == 0:
            print('Iteration: ', iteration)       
            T.save(net_d1.state_dict(), path_work + 'net_d1.pt')
            T.save(net_d1.state_dict(), path_work + '备用net_d1.pt')
            T.save(optimizer_d1.state_dict(), path_work + 'optimizer_d1.pt')
            T.save(optimizer_d1.state_dict(), path_work + '备用optimizer_d1.pt')
            T.save(net_g1.state_dict(), path_work + 'net_g1.pt')
            T.save(net_g1.state_dict(), path_work + '备用net_g1.pt')
            T.save(optimizer_g1.state_dict(), path_work + 'optimizer_g1.pt')
            T.save(optimizer_g1.state_dict(), path_work + '备用optimizer_g1.pt')
            T.save(net_d2.state_dict(), path_work + 'net_d2.pt')
            T.save(net_d2.state_dict(), path_work + '备用net_d2.pt')
            T.save(optimizer_d2.state_dict(), path_work + 'optimizer_d2.pt')
            T.save(optimizer_d2.state_dict(), path_work + '备用optimizer_d2.pt')
            T.save(net_g2.state_dict(), path_work + 'net_g2.pt')
            T.save(net_g2.state_dict(), path_work + '备用net_g2.pt')
            T.save(optimizer_g2.state_dict(), path_work + 'optimizer_g2.pt')
            T.save(optimizer_g2.state_dict(), path_work + '备用optimizer_g2.pt')
            np.save(path_work + 'iteration.npy', iteration) 
            show(net_g1, net_g2)    
            print('These iterations cost {} seconds'.format(time.time() - time_start))
            time_start = time.time()

In [ ]:
# train(iterations=1, load_model=False)

In [ ]:
# train(iterations=iterations, load_model=True)

In [ ]:
net_g1 = Generator().to(device)
net_g1.load_state_dict(T.load(path_work + 'net_g1.pt'))
net_g2 = Generator().to(device)
net_g2.load_state_dict(T.load(path_work + 'net_g2.pt'))
show(net_g1, net_g2)